# CRPS reference based on the CCPA climatology

The notebook computes the CRPS reference by using the CCPA grid-point-wise CDFs 

In [1]:
import os
import sys
import time
import h5py
import numpy as np
from glob import glob
from datetime import datetime, timedelta

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du
import verif_utils as vu

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
with h5py.File(save_dir+'CCPA_domain.hdf', 'r') as h5io:
    lon_CCPA = h5io['lon_CCPA'][...]
    lat_CCPA = h5io['lat_CCPA'][...]
    land_mask_CCPA = h5io['land_mask_CCPA'][...]
land_mask_ = land_mask_CCPA==1
grid_shape = land_mask_.shape

### 2002-2019 CDFs

In [5]:
with h5py.File(camp_dir+'CCPA/CCPA_CDFs_2002_2019.hdf', 'r') as h5io:
    CCPA_CDFs = h5io['CCPA_CDFs'][...]

q_bins = np.array([0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
                   0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21,
                   0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32,
                   0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43,
                   0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54,
                   0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65,
                   0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
                   0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
                   0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 
                   0.99, 0.991, 0.992, 0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999, 1.0])

### CRPS reference for CCPA 2021

In [6]:
filename_ccpa = camp_dir+'CCPA/CCPA_lead_y{}.hdf'
with h5py.File(filename_ccpa.format(2021), 'r') as h5io:
    CCPA = h5io['CCPA_lead'][...]
    
LEADs = np.arange(6, 144+6, 6)
N_leads = len(LEADs)

In [1]:
CRPS_ref = np.empty((N_leads,)+grid_shape)
CRPS_ref[...] = np.nan
# for ix in range(grid_shape[0]):
    # for iy in range(grid_shape[1]):
    #     if land_mask_[ix, iy]:
for ilead, lead in enumerate(LEADs):
    ind_hour = lead % 24
    ind_hour = int(ind_hour/6)
    
    ccpa_cdf = CCPA_CDFs[:, ..., ind_hour]
    ccpa_obs = CCPA[:, ilead, ...]
    
    crps_ = vu.CRPS_1d_from_quantiles(q_bins, ccpa_cdf[:, land_mask_], ccpa_obs[:, land_mask_])
    CRPS_ref[ilead, land_mask_] = np.nanmean(crps_, axis=0)
    print('lead = {:02} done'.format(lead))

CRPS_ref[:, land_mask_CCPA==0] = np.nan

# backup every lead time  
tuple_save = (CRPS_ref,)
label_save = ['CRPS_ref',]
du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_CCPA_2021.hdf')